In [16]:
import numpy as np

R = np.array([[0.5076271,  0.3434859,  0.7901469],
  [ 0.8382091,  0.0152542, -0.5451356],
  [-0.1992995,  0.9390339, -0.2801695]])

if (R[2,2] < 0):
    if (R[0,0] >R[1,1]):
        t = 1 + R[0,0] -R[1,1] -R[2,2]
        q = np.array([t, R[0,1]+R[1,0], R[2,0]+R[0,2], R[1,2]-R[2,1]])
    else:
        t = 1 -R[0,0] + R[1,1] -R[2,2]
        q = np.array([R[0,1]+R[1,0], t, R[1,2]+R[2,1], R[2,0]-R[0,2]])
else:
    if (R[0,0] < -R[1,1]):
        t = 1 -R[0,0] -R[1,1] + R[2,2]
        q = np.array([R[2,0]+R[0,2], R[1,2]+R[2,1], t, R[0,1]-R[1,0]])
    else:
        t = 1 + R[0,0] + R[1,1] + R[2,2]
        q = np.array([R[1,2]-R[2,1], R[2,0]-R[0,2], R[0,1]-R[1,0], t])
q *= 0.5 / np.sqrt(t)
q

array([ 0.66568431,  0.44378956,  0.22189474, -0.55738489])

In [17]:
from scipy.spatial.transform import Rotation as ScipyRot
plot_lambda_bar = []
rot = ScipyRot.from_matrix(R)
print(rot.as_quat())

[0.66568432 0.44378957 0.22189475 0.5573849 ]
